## Create a picture with hill climbing using a mosaic grid

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cv2
import numpy as np
import helper
from individual import Individual

In [ ]:
img = cv2.imread("./img/bird.jpg")

In [ ]:
helper.show_img(img)

In [ ]:
def get_masks(shape):
    mid_coord = (shape[1]//2, shape[0]//2)
    tl = (0, 0)
    tr = (shape[1], 0)
    bl = (0, shape[0])
    br = (shape[1], shape[0])
    
    top = np.array((tl, mid_coord, tr))
    top = top.reshape((-1, 1, 2))
    
    right = np.array((tr, mid_coord, br))
    right = right.reshape((-1, 1, 2))
    
    bottom = np.array((bl, mid_coord, br))
    bottom = bottom.reshape((-1, 1, 2))
    
    left = np.array((tl, mid_coord, bl))
    left = left.reshape((-1, 1, 2))
    
    top_m = np.zeros(shape)
    cv2.fillPoly(top_m, [top], (1, 0, 0))
    top_m = np.sum(top_m, axis=2).astype(bool)

    right_m = np.zeros(shape)
    cv2.fillPoly(right_m, [right], (1, 0, 0))
    right_m = np.sum(right_m, axis=2).astype(bool)

    bottom_m = np.zeros(shape)
    cv2.fillPoly(bottom_m, [bottom], (1, 0, 0))
    bottom_m = np.sum(bottom_m, axis=2).astype(bool)
    
    left_m = np.zeros(shape)
    cv2.fillPoly(left_m, [left], (1, 0, 0))
    left_m = np.sum(left_m, axis=2).astype(bool)
    
    return top_m, right_m, bottom_m, left_m

In [ ]:
def build_square(img):
    masks = get_masks(img.shape)
    for mask in masks:
        color = np.average(img[mask], axis=0)
        img[mask, :] = color.astype(np.uint)
    return img

def build_mosaic(img, grid_size):
    canvas = np.zeros_like(img)
    
    # grid size is (rows, columns)
    r_size = img.shape[0] // grid_size[0]
    c_size = img.shape[1] // grid_size[1]
    
    for r_n in range(grid_size[0]):
        for c_n in range(grid_size[1]):
            # starting coordinates for row/column
            r_s = r_n * r_size
            c_s = c_n * c_size            
            
            # end coordinates for row/column
            if r_n < grid_size[0] - 1:
                r_e = r_s + r_size
            else:
                r_e = img.shape[0]
            if c_n < grid_size[1] - 1:
                c_e = c_s + c_size
            else:
                c_e = img.shape[1]                
            
            canvas[r_s:r_e, c_s:c_e] = build_square(img[r_s:r_e, c_s:c_e])
    
    return canvas

In [ ]:
res = build_mosaic(img, (14, 16))
helper.show_img(res)
cv2.imwrite("./mosaic.png", res)

In [ ]:
grid_rows = 12
grid_cols = 12
ind = Individual(caps, (grid_rows,grid_cols))
ind.draw()
helper.show_img(ind.canvas)

In [ ]:
im_name = "bird"
cv2.imwrite("./climb/{}_r{}_c{}_i{:0>3}.png".format(im_name, grid_rows, grid_cols, 0), ind.canvas)
for i in range(120):
    print("Iteration: {}".format(i+1))
    ind.mutate()
    cv2.imwrite("./climb/{}_r{}_c{}_i{:0>3}.png".format(im_name, grid_rows, grid_cols, i+1), ind.canvas)

In [ ]:
helper.show_img(ind.canvas)